In [1]:
import os
from hmmlearn import hmm
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import classification_report, confusion_matrix
import random

In [2]:
def load_annotated_sequences_from_paths(filepaths):
    sequences = []
    for filepath in filepaths:
        with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
            sequence = []
            for line in f:
                tokens = line.strip().split()
                sequence.extend(tokens[::-1])
            sequences.append(sequence)
    return sequences

In [3]:
# Apenas para obter os ficheiros .il de uma pasta (path relativo)
def get_il_files(pasta):
    return [os.path.join(pasta, f) for f in os.listdir(pasta) if f.endswith(".il") and os.path.isfile(os.path.join(pasta, f))]

In [4]:
# Das pastas divididas por similaridade
files_train_vuln = get_il_files("../data/il_train_test/train_com_id/vuln")
files_train_nao_vuln = get_il_files("../data/il_train_test/train_com_id/nvuln")

files_test_vuln = get_il_files("../data/il_train_test/test_com_id/vuln")
files_test_nao_vuln = get_il_files("../data/il_train_test/test_com_id/nvuln")

In [5]:
random.seed(1)
random.shuffle(files_train_vuln)
random.shuffle(files_train_nao_vuln)

In [6]:
# Ler os ficheiros e obter todos os tokens
train_seqs_vuln = load_annotated_sequences_from_paths(files_train_vuln)
train_seqs_nvuln = load_annotated_sequences_from_paths(files_train_nao_vuln)

all_sequences = train_seqs_vuln + train_seqs_nvuln
all_tokens = [token for seq in all_sequences for token in seq]

In [7]:
# Encoder porque o modelo apenas suporta números
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(np.array(all_tokens).reshape(-1, 1))

import joblib
joblib.dump(encoder, "ordinal_encoder.pkl")

def encode_sequences(sequences):
    # sequences deve ser uma lista de listas de strings
    encoded = []
    for seq in sequences:
        arr = np.array(seq).reshape(-1, 1)
        encoded_seq = encoder.transform(arr).astype(int)

        # Remover tokens desconhecidos (-1)
        encoded_seq = encoded_seq[encoded_seq != -1].reshape(-1, 1)

        if len(encoded_seq) > 0:
            encoded.append(encoded_seq)

    return encoded

X_vuln = encode_sequences(train_seqs_vuln)
X_nao_vuln = encode_sequences(train_seqs_nvuln)

In [8]:

def treinar_hmm(sequences, n_states=2, alpha=0.01):
    X_concat = np.concatenate(sequences)
    lengths = [len(seq) for seq in sequences]

    model = hmm.CategoricalHMM(n_components=n_states, n_iter=100, random_state=42)
    model.fit(X_concat, lengths)

    # Suavização de Laplace segura (sem mudar shape!)
    model.emissionprob_ += alpha
    model.emissionprob_ /= model.emissionprob_.sum(axis=1, keepdims=True)

    model.transmat_ += alpha
    model.transmat_ /= model.transmat_.sum(axis=1, keepdims=True)

    return model


modelo_vuln = treinar_hmm(X_vuln, alpha=0.01)
modelo_nao_vuln = treinar_hmm(X_nao_vuln, alpha=0.01)

def prever_sql_injection(filepath):
    with open(filepath, 'r') as f:
        tokens = [tok for line in f for tok in line.strip().split()][::-1]

    arr = np.array(tokens).reshape(-1, 1)
    seq = encoder.transform(arr).astype(int)

    # Remover valores fora do intervalo [0, num_observations - 1]
    max_index = modelo_vuln.emissionprob_.shape[1]  # ou modelo_nao_vuln, ambos devem ter mesmo N
    seq = seq[(seq >= 0) & (seq < max_index)].reshape(-1, 1)

    if len(seq) == 0:
        print(f"[AVISO] Nenhum token válido em {filepath}.")
        return "não vulnerável"

    score_vuln = modelo_vuln.score(seq)
    score_nao_vuln = modelo_nao_vuln.score(seq)

    # Probabilidades normalizadas
    prob_vuln = np.exp(score_vuln)
    prob_nao_vuln = np.exp(score_nao_vuln)
    total = prob_vuln + prob_nao_vuln
    prob_vuln /= total
    prob_nao_vuln /= total

    print(f"Probabilidades normalizadas:")
    print(f"  Vulnerável:     {prob_vuln:.4f}")
    print(f"  Não vulnerável: {prob_nao_vuln:.4f}")

    return "vulnerável" if prob_vuln > prob_nao_vuln else "não vulnerável"


In [9]:
y_true = ['vulnerável'] * len(files_test_vuln) + ['não vulnerável'] * len(files_test_nao_vuln)
y_pred = [prever_sql_injection(f) for f in files_test_vuln + files_test_nao_vuln]

print("Classification Report:\n")
print(classification_report(y_true, y_pred, digits=5))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nAccuracy:", np.mean(np.array(y_true) == np.array(y_pred)))

Probabilidades normalizadas:
  Vulnerável:     0.9737
  Não vulnerável: 0.0263
Probabilidades normalizadas:
  Vulnerável:     0.9738
  Não vulnerável: 0.0262
Probabilidades normalizadas:
  Vulnerável:     0.9951
  Não vulnerável: 0.0049
Probabilidades normalizadas:
  Vulnerável:     0.9846
  Não vulnerável: 0.0154
Probabilidades normalizadas:
  Vulnerável:     0.8850
  Não vulnerável: 0.1150
Probabilidades normalizadas:
  Vulnerável:     0.9678
  Não vulnerável: 0.0322
Probabilidades normalizadas:
  Vulnerável:     0.6229
  Não vulnerável: 0.3771
Probabilidades normalizadas:
  Vulnerável:     0.6779
  Não vulnerável: 0.3221
Probabilidades normalizadas:
  Vulnerável:     0.9892
  Não vulnerável: 0.0108
Probabilidades normalizadas:
  Vulnerável:     0.9965
  Não vulnerável: 0.0035
Probabilidades normalizadas:
  Vulnerável:     0.8450
  Não vulnerável: 0.1550
Probabilidades normalizadas:
  Vulnerável:     0.9993
  Não vulnerável: 0.0007
Probabilidades normalizadas:
  Vulnerável:     0.989

In [10]:
# Matrizes do modelo vulneravel
print("\nHMM Vulnerável Transition Matrix:")
print(modelo_vuln.transmat_)
print("\nHMM Vulnerável Emission Matrix:")
print(modelo_vuln.emissionprob_)
print("\nHMM Vulnerável Start Probabilities:")
print(modelo_vuln.startprob_)


HMM Vulnerável Transition Matrix:
[[0.97566294 0.02433706]
 [0.99019608 0.00980392]]

HMM Vulnerável Emission Matrix:
[[0.00608255 0.00608255 0.00608255 0.00690267 0.00660743 0.00647621
  0.00650902 0.00831327 0.00900816 0.00608255 0.005      0.005
  0.0197621  0.00904096 0.00877254 0.00500014 0.005      0.03452421
  0.01077362 0.00500007 0.00604975 0.00821486 0.00604975 0.01290593
  0.005      0.00585292 0.01159374 0.00877254 0.0078212  0.005
  0.005      0.01290593 0.00608255 0.0197621  0.0197621  0.0197621
  0.0197621  0.0197621  0.00660743 0.0197621  0.00877254 0.00585292
  0.005      0.0064106  0.0197621  0.0078212  0.00608255 0.00608255
  0.01159374 0.00877254 0.005      0.00890376 0.00608255 0.00601694
  0.00582012 0.03993011 0.00595134 0.005      0.00716511 0.005
  0.005      0.00618097 0.005      0.005      0.00585292 0.00729633
  0.00585292 0.03624868 0.005      0.01261068 0.00588573 0.00677145
  0.00588573 0.00591853 0.00536085 0.03780467 0.0078212  0.005
  0.04154441 0.005

In [11]:
# Matrizes do modelo não vulneravel
print("\nHMM Não Vulnerável Transition Matrix:")
print(modelo_nao_vuln.transmat_)
print("\nHMM Não Vulnerável Emission Matrix:")
print(modelo_nao_vuln.emissionprob_)
print("\nHMM Não Vulnerável Start Probabilities:")
print(modelo_nao_vuln.startprob_)


HMM Não Vulnerável Transition Matrix:
[[0.34516315 0.65483685]
 [0.99019608 0.00980392]]

HMM Não Vulnerável Emission Matrix:
[[0.00620454 0.00620454 0.00620454 0.00497512 0.00497512 0.00497512
  0.00497512 0.00497512 0.00497512 0.00497518 0.00497512 0.00497512
  0.00497512 0.00497512 0.01736384 0.00497512 0.00497512 0.00497512
  0.00639368 0.00497512 0.00634639 0.00497512 0.00497512 0.00497512
  0.00497512 0.00497512 0.00497512 0.01551972 0.01244618 0.00960907
  0.00606268 0.00652076 0.00497512 0.00497512 0.00497512 0.00497512
  0.00497512 0.00497512 0.01003757 0.00497512 0.01736384 0.00559817
  0.00497512 0.00497512 0.02625345 0.00497512 0.00497512 0.00497512
  0.02871228 0.01736383 0.00497512 0.01708013 0.00497512 0.00644096
  0.00497512 0.06394112 0.00497512 0.00497512 0.0082378  0.00497512
  0.00521155 0.00663011 0.00582626 0.00544798 0.00644096 0.00790681
  0.00497512 0.05747813 0.00497512 0.00951246 0.00639368 0.00659559
  0.00639368 0.00639368 0.00497512 0.05027432 0.00497512 

In [12]:
import json

# Preparar modelo como um dicionário serializável
model_nvuln = {
    "start_probs": modelo_nao_vuln.startprob_.tolist(),
    "trans_probs": modelo_nao_vuln.transmat_.tolist(),
    "emit_probs": modelo_nao_vuln.emissionprob_.tolist(),
}

model_vuln = {
    "start_probs": modelo_vuln.startprob_.tolist(),
    "trans_probs": modelo_vuln.transmat_.tolist(),
    "emit_probs": modelo_vuln.emissionprob_.tolist(),
}

# Guardar modelo em JSON
with open("hmm_unsup_nvuln.json", "w") as f:
    json.dump(model_nvuln, f)

# Guardar modelo em JSON
with open("hmm_unsup_vuln.json", "w") as f:
    json.dump(model_vuln, f)

print("Modelo HMM guardado.")

Modelo HMM guardado.
